# Week 05 - Nearest Neighbor and Locality-Sensitive Hashing
**Computational Tools for Data Science - Fall, 2022**

In [1]:
import numpy as np

## Exercise 2 - `shingle` function

**Implement a function `shingle` that take an integer `q` and a string and produces a list of shingles,
where each shingle is a list of `q` words.**

HINT: Here, the input string (a document in the case of the test files) would
consist of separate words. The $q$-shingles for this string would be all of the sequences
of $q$ consecutive words in the string. 

For example, if the string was
`‘i want to go home do you want to’` and $q = 2$, then the q-shingles would be
`{i want, want to, to go, go home, home do, do you, you want}`. 

Note that we do not include
`‘want to’` twice because we only care about the set of shingles, we do not care about the
order. Of course it will probably be output as a list which does have an order but there should
not be repeats. 

For more details, see Section 3.2.1 and 3.2.2 in the online version of the book. Note that in the
book they use sequences of $q$ characters rather than words, but the principle is the same.

Extract from the book, to recall concepts (section 3.2.1):

> A document is a string of characters. Define a k-shingle for a document to be
any substring of length k found within the document. Then, we may associate
with each document the set of k-shingles that appear one or more times within
that document.

> Example 3.3 : Suppose our document D is the string abcdabd, and we pick
k = 2. Then the set of 2-shingles for D is {ab, bc, cd, da, bd}.
Note that the substring ab appears twice within D, but appears only once
as a shingle. A variation of shingling produces a bag, rather than a set, so each
shingle would appear in the result as many times as it appears in the document.
However, we shall not use bags of shingles here.

> There are several options regarding how white space (blank, tab, newline,
etc.) is treated. It probably makes sense to replace any sequence of one or more
white-space characters by a single blank. That way, we distinguish shingles that
cover two or more words from those that do not.

> Example 3.4 : If we use k = 9, but eliminate whitespace altogether, then we
would see some lexical similarity in the sentences “The plane was ready for
touch down”. and “The quarterback scored a touchdown”. However, if we
retain the blanks, then the first has shingles touch dow and ouch down, while
the second has touchdown. If we eliminated the blanks, then both would have
touchdown.

In [2]:
def shingle(aString, q, delimiter=' '):
    """
    Input:
        - aString (str): string to split into shingles
        - q (int)
        - delimiter (str): string of the delimiter to consider to split the input string (default: space)
    Return: list of unique shingles
    """
    all_shingles = []
    if delimiter != '':
        words_list = aString.split(delimiter)
    else:
        words_list = aString
    for i in range (len(words_list)-q+1):
        all_shingles.append(delimiter.join(words_list[i:i+q]))
    return list(set(all_shingles))

In [11]:
# Example from the Book
ex_string, q = 'abcdabd', 2
ex_shingles = shingle(ex_string, q, delimiter='')
print('Initial string:', ex_string)
print(f'>> Shingles with q = {q} :',ex_shingles)

# Example from the HINT
ex_string, q = 'i want to go home do you want to', 2
ex_shingles = shingle(ex_string, q)
assert len(ex_shingles) == 7
print('\nInitial string:', ex_string)
print(f'>> Shingles with q = {q} :',ex_shingles)

Initial string: abcdabd
>> Shingles with q = 2 : ['bc', 'cd', 'ab', 'da', 'bd']

Initial string: i want to go home do you want to
>> Shingles with q = 2 : ['home do', 'you want', 'do you', 'go home', 'to go', 'i want', 'want to']


## Exercise 3 - Minhashing

**3.1. Implement a minhash algorithm `minhash` that takes a list of shingles and a seed for the hash function
mapping the shingles, and outputs the minhash. Feel free to use the `listhash` function in the template.**

References to the Book:
- See Chapter 3, Section 3.2.3 for more details about hashing shingles.
- See Chapter 3, Section 3.3.2 for more details about minhashing.
- See Chapter 3, Section 3.3.3

HINT: Given a hash function $h$ that maps to integers, the minhash $h_{\text{min}}$ of a set of
elements $S$ is the minimum value that the function $h$ takes on the set $S$, i.e., $h_{\text{min}}(S) = \min \{ h(S) : s \in S \}$. In this case the exercise suggests to use the `listhash` function provided. So the minhash function should take a list of shingles (and a seed for `listhash`) and compute the minhash for this list, i.e., it should compute
`listhash` of every shingle in the list and return the minimum value.

In [12]:
from similarity import listhash

In [13]:
def minhash(shingles_list, seed):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - seed (int): seed for listhash function
    Return: minhash of given shingles
    """
    minhash_value = None
    for aShingle in shingles_list:
        hashcode = listhash([aShingle], seed)
        if minhash_value == None or hashcode < minhash_value:
            minhash_value = hashcode
    return minhash_value

In [16]:
print(f'MinHash of {ex_shingles}:', minhash(ex_shingles, 42))

MinHash of ['home do', 'you want', 'do you', 'go home', 'to go', 'i want', 'want to']: -2058595977


**3.2. Extend the minhash algorithm to output k different minhashes in an array. Use different seeds for each
minhash, e.g., 1, . . . , k.**

HINT: For the second part, just add $k$ as input and return a sequence of $k$ minhashes, each using a different seed for the hash function. Note that we care about the order of the minhashes here, because we will be comparing the values of the i-th minhash of a given list of shingles with the i-th minhash of another list of shingles.

In [17]:
def minhash2(shingles_list, k):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - k (int): seed for listhash function
    Return: sequence of k minhashes
    """
    all_minhash = []
    for i in range(k):
        all_minhash.append(minhash(shingles_list, i))
    return all_minhash

In [23]:
k=10
print(f'MinHash of {ex_shingles} with k = {k}:\n', minhash2(ex_shingles, k))

MinHash of ['home do', 'you want', 'do you', 'go home', 'to go', 'i want', 'want to'] with k = 10:
 [-1444075960, -1361643506, -1787085820, -773815373, -1475701122, -1751671059, -1905760484, -1765166633, -2127236308, -1658139371]


## Exercise 4 - Signatures

**Construct a function `signatures` that takes the `docs` dictionary and outputs a new dictionary
consisting of document id’s as keys and signatures as values.**

HINT: This should take each document and convert it to a list of $q$-shingles (so $q$ should
be an input to the function) and then use the function from 3.2 to convert that list of shingles
for that document to a sequence of $k$ minhash values (so $k$ will also be an input) which is
known as the signature of that document. See Section 3.3.4 (page 83) for info on minhash
signatures.

Note that it is not necessarily most efficient to simply compute the signature for each document
separately. In the book in Section 3.3.5 (page 84), they give an algorithm for computing
the signatures of a collection of sets/documents simultaneously. Let $U$ be the set of all shingles produced from a given set $D$
of documents and let shingle($d$) denote the list of shingles produced from a given document
$d \in D$ (assume that these have already been computed prior to running the below code).
Suppose we are creating signatures using minhashes from hash functions $h_1, ..., h_k$. The following
creates a matrix SIG with rows indexed by $1, ... , k$, and columns indexed by $D$ such
that the $d$ column is the signature of document $d$.

![pseudocode](https://www.zupimages.net/up/22/44/gcp6.png)

In [24]:
import os

def clean_text(aString):
    output = aString.replace('\n','')
    output_list = output.split()
    output_list = [''.join(ch for ch in aWord if ch.isalnum()) for aWord in output_list]
    output_list = [s.lower() for s in output_list]
    output = ' '.join(output_list)
    return " ".join(output.split())

q = 3 # length of shingle
k = 100 # number of minhashes
docs = {} #dictionary mapping document id to document contents

# read data sets
srcfolder = os.path.abspath('.')
datafolder = os.path.join(srcfolder, "ats_corpus_small")   # change to ats_corpus for large data set

for file in os.listdir(datafolder):
    filepath = os.path.join(datafolder, file)
    f = open(filepath, 'r')
    docs[file] = f.read()
    docs[file] = clean_text(docs[file])
    print ("read document " + file)
    f.close()

read document calltounconv00baxt.txt
read document gospeltruth00whit.txt
read document lifeofrevrichard00baxt.txt
read document memoirjamesbrai00ricegoog.txt
read document practicalthought00nev.txt
read document remember00palm.txt
read document remembermeorholy00palm.txt
read document thoughtsonpopery00nevi.txt


In [26]:
def signature(dict_docs, q = q, num_hashes = k):
    """
    Input:
        - dict_docs (dict of str:str): dictionary of {title:document}
        - q (int)
        - num_hashes (int)
    Return: dictionary consisting of document id’s as keys and signatures as values
    """
    dict_signatures = {}
    total_texts = len(list(dict_docs.keys()))
    counter = 1
    for key,text in dict_docs.items():
        print(f'{counter}/{total_texts} - {key} - Processing...')
        doc_shingles = shingle(text, q)
        minhash_values = minhash2(doc_shingles, num_hashes)
        dict_signatures[key] = minhash_values
        counter += 1
    return dict_signatures

In [27]:
dict_signatures_docs = signature(docs)

1/8 - calltounconv00baxt.txt - Processing...
2/8 - gospeltruth00whit.txt - Processing...
3/8 - lifeofrevrichard00baxt.txt - Processing...
4/8 - memoirjamesbrai00ricegoog.txt - Processing...
5/8 - practicalthought00nev.txt - Processing...
6/8 - remember00palm.txt - Processing...
7/8 - remembermeorholy00palm.txt - Processing...
8/8 - thoughtsonpopery00nevi.txt - Processing...


## Exercise 5 - Jaccard Similarity

**Implement a function `jaccard` that takes two document names and outputs the estimated
Jaccard similarity using signatures.**

HINT: Given two sets $S$ and $T$, the Jaccard similarity of $S$ and $T$ is equal to $\frac{|S ∩
T|}{|S ∪ T|}$, i.e., the size of the intersection divided by the size of the union of the two sets
(see Section 3.1.1, page 74). For a random *injective* hash function $h$, the probability that
$h_{\text{min}}(S) = h_{\text{min}}(T)$ is equal to the Jaccard similarity of $S$ and $T$ (see Section 3.3.3, page 83).
The hash functions used in practice won’t necessarily be injective, but assuming there are not
many collisions we can still use this as an estimation of Jaccard similarity. So this function
should take two documents, convert each to a list of $q$-shingles (so $q$ should be an input) and
then use the function from 3.2 to convert these into signatures consisting of $k$ minhash values
(so $k$ will be an input). Then it estimates the Jaccard similarity of the two documents by
counting how many positions of the two signatures agree and then dividing that by $k$.

In [36]:
def jaccard(name1, name2, signatures_dict):
    """
    Input:
        - name1 (str): key of the first document S
        - name2 (str): key of the second document T
        - signatures_dict (dict of str:list): dictionary of signatures
    Return: Jaccard similarity between S and T
    """
    signatures_doc1 = np.array(signatures_dict[name1])
    signatures_doc2 = np.array(signatures_dict[name2])
    return len(np.intersect1d(signatures_doc1, signatures_doc2))/len(np.union1d(signatures_doc1, signatures_doc2))

In [38]:
first_doc_key = list(docs.keys())[0]
second_doc_key = list(docs.keys())[1]
print(f'Jaccard similarity between {first_doc_key} and {second_doc_key}:', jaccard(first_doc_key, second_doc_key, dict_signatures_docs))

Jaccard similarity between calltounconv00baxt.txt and gospeltruth00whit.txt: 0.005025125628140704


## Exercise 6 - Find Similar Items

**Implement a function `similar` that finds all pairs of documents whose estimated Jaccard
similarity is ≥ 0.6. Test your program for different values of $k$ and $q$. Compare your results for most similar
documents with your own visual impression of the similarity of files.**

HINT: This just combines techniques from previous questions. You can use the function
from Exercise 4 to convert all the documents to signatures, and then write a function that
computes Jaccard similarity based on signatures (essentially the second half of the function
from Exercise 5). Then compute/estimate the Jaccard similarity for all pairs of documents
and return those with Jaccard similarity at least 0.6.

In [39]:
def similar(signatures_dict, jaccard_threshold=0.6):
    """
    Input:
        - signatures_dict (dict of str:list): dictionary of signatures
        - jaccard_threshold (float): lower bound for Jaccard similarity score to consider
            two documents as similar
    Return: dictionary of similar items
    """
    list_keys = list(signatures_dict.keys())
    similar_items = {}
    for i in range (len(list_keys)-1):
        for j in range (i+1, len(list_keys)):
            similarity_score = jaccard(list_keys[i], list_keys[j], signatures_dict)
            if similarity_score >= jaccard_threshold:
                similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items

In [50]:
found_similar_items = similar(dict_signatures_docs)
print('Found similar items:\n', found_similar_items)

Found similar items:
 {('remember00palm.txt', 'remembermeorholy00palm.txt'): 0.6129032258064516}


## Exercise 7 - Locality-Sensitive Hashing

**Use locality-sensitive hashing to speed up your solution to the find similar item
exercise.**

HINT: Here, we take our signatures for all of our documents and break them up into $b$
blocks of length $r$ (so $k = br$ where k is the length of the signatures). For each block, we apply
a hash function to that part of the signature for each of the documents, and if two documents
get hashed to the same value for at least one of the blocks, then they will be considered a
candidate for being similar. We then compute the Jaccard similarity of only the candidate
pairs using their full signatures. Note that the values of $b$ and $r$ need to be chosen correctly so
that potential similar pairs are correctly identified. If one wants to identify pairs with Jaccard
similarity at least some value $s$, then one should choose $b$ and $r$ such that $(\frac{1}{b})^{\frac{1}{r}} ≈ s$. See
Section 3.4. (page 91) in the book.

In [45]:
import mmh3

In [46]:
b,r = 20, 5
assert k == b*r

def lsh(signatures_dict, jaccard_threshold=0.6, seed=42):
    lsh_dict = {}
    for key, values in signatures_dict.items():
        blocks = np.split(np.array(values), b)
        blocks_hash_values = []
        for aBlock in blocks:
            blocks_hash_values.append(mmh3.hash(aBlock, seed))
        lsh_dict[key] = blocks_hash_values
    list_keys = list(lsh_dict.keys())
    similar_items = {}
    for i in range (len(list_keys)-1):
        for j in range (i+1, len(list_keys)):
            common_values = np.intersect1d(lsh_dict[list_keys[i]], lsh_dict[list_keys[j]])
            if len(common_values) > 0:
                # we found a candidate
                similarity_score = jaccard(list_keys[i], list_keys[j], signatures_dict)
                if similarity_score >= jaccard_threshold:
                    similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items

In [49]:
found_similar_items_with_lsh = lsh(dict_signatures_docs)
print('Found similar items with LSH:\n', found_similar_items_with_lsh)

Found similar items with LSH:
 {('remember00palm.txt', 'remembermeorholy00palm.txt'): 0.6129032258064516}


# THE END.